# **Random Forest**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/NotAnnotated/Regular/NotAveragePooled/split_features_6s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [4]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [5]:
train_data

{'melspectrogram': array([[[-1.73638668e-02,  0.00000000e+00, -8.11987277e-03, ...,
          -1.67526044e-02, -5.46729891e-03, -8.64551112e-04],
         [-1.93848714e-01, -2.12280467e-01, -2.69487441e-01, ...,
          -3.62844795e-01, -2.54802048e-01, -2.05738589e-01],
         [-2.68348396e-01, -2.86629409e-01, -3.42423439e-01, ...,
          -4.13956523e-01, -3.27738762e-01, -2.80380309e-01],
         ...,
         [-9.98975337e-01, -9.98841584e-01, -9.96024787e-01, ...,
          -9.24670994e-01, -9.80563939e-01, -9.91996884e-01],
         [-9.99578595e-01, -9.99459684e-01, -9.97734010e-01, ...,
          -9.48356807e-01, -9.88852501e-01, -9.95917022e-01],
         [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
          -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]],
 
        [[-1.72525998e-02,  0.00000000e+00, -8.14473629e-03, ...,
          -1.66835915e-02, -5.44226915e-03, -8.53342586e-04],
         [-1.92598045e-01, -2.11414754e-01, -2.70316750e-01, ...,
   

In [6]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [7]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [8]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [9]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [10]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [11]:
display(train_y.shape)
display(train_y[:15])

(6543,)

array([19,  2,  7,  1,  2, 19,  7,  7,  0, 18, 19, 12,  0, 15, 12],
      dtype=int32)

In [12]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(6543, 128, 517)

array([[-1.73640419e-02,  0.00000000e+00, -8.11868720e-03, ...,
        -1.66713316e-02, -5.46701998e-03, -8.66253627e-04],
       [-1.93848863e-01, -2.12279290e-01, -2.69453466e-01, ...,
        -3.61078233e-01, -2.54836142e-01, -2.05884218e-01],
       [-2.68348634e-01, -2.86627918e-01, -3.42380673e-01, ...,
        -4.11952227e-01, -3.27786148e-01, -2.80581295e-01],
       ...,
       [-9.98977244e-01, -9.99002397e-01, -9.99078691e-01, ...,
        -9.98179734e-01, -9.98767555e-01, -9.98777688e-01],
       [-9.99582410e-01, -9.99592662e-01, -9.99624014e-01, ...,
        -9.99252260e-01, -9.99510348e-01, -9.99518573e-01],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

'mfcc'

(6543, 20, 517)

...

(6543, 84, 517)

array([[ -6.15837097,  -6.18668032,  -6.33215666, ...,  -6.40537167,
         -6.21032095,  -6.11683273],
       [ -6.38547564,  -6.46231508,  -6.60910749, ...,  -6.68341064,
         -6.48439455,  -6.36775827],
       [ -6.6979661 ,  -6.71482658,  -6.91855526, ...,  -6.98905373,
         -6.77421951,  -6.61743736],
       ...,
       [-25.96397591, -80.        , -80.        , ..., -52.15293503,
        -60.1878624 , -55.97823334],
       [-26.20120621, -80.        , -80.        , ..., -53.98522568,
        -72.95877838, -61.09448624],
       [-26.3871727 , -80.        , -80.        , ..., -58.57253265,
        -58.14158249, -72.96665192]])

'id'

(6543,)

'XC280984.mp3'

In [13]:
display(val_y.shape)
display(val_y[:15])

(2028,)

array([12,  7, 19,  3,  7, 13,  9, 11, 11, 12, 17,  2, 17,  3, 18],
      dtype=int32)

In [14]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(2028, 128, 517)

array([[-1.73589345e-02,  0.00000000e+00, -8.13421980e-03, ...,
        -1.65349785e-02, -5.42709185e-03, -9.02378582e-04],
       [-1.93414882e-01, -2.12432668e-01, -2.69882023e-01, ...,
        -3.59408289e-01, -2.53411531e-01, -2.04721957e-01],
       [-2.68646866e-01, -2.87940741e-01, -3.44254375e-01, ...,
        -4.12034959e-01, -3.27018231e-01, -2.79676229e-01],
       ...,
       [-9.95157480e-01, -9.97765958e-01, -9.95613873e-01, ...,
        -9.57726657e-01, -9.91686523e-01, -9.96120214e-01],
       [-9.96097267e-01, -9.98655617e-01, -9.96654451e-01, ...,
        -9.73076999e-01, -9.91579652e-01, -9.93710876e-01],
       [-9.95531142e-01, -9.98879671e-01, -1.00000000e+00, ...,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00]])

'mfcc'

(2028, 20, 517)

...

(2028, 84, 517)

array([[-9.86957550e-03, -3.93637419e-02, -1.85878515e-01, ...,
        -3.02529573e-01, -1.00206137e-01,  1.19209290e-07],
       [-2.52804160e-01, -3.36061716e-01, -4.90267754e-01, ...,
        -5.03781438e-01, -3.05644274e-01, -1.89857244e-01],
       [-5.24746299e-01, -5.40174484e-01, -7.42211819e-01, ...,
        -7.53739119e-01, -5.42865753e-01, -3.89395595e-01],
       ...,
       [-1.99781590e+01, -5.45793953e+01, -4.25640640e+01, ...,
        -3.99915886e+01, -4.05000114e+01, -4.03554115e+01],
       [-2.02033577e+01, -4.82693367e+01, -3.77919197e+01, ...,
        -5.28779030e+01, -4.72922096e+01, -3.86849785e+01],
       [-2.01529293e+01, -4.67737961e+01, -3.82349358e+01, ...,
        -5.10622139e+01, -4.05895081e+01, -4.18449020e+01]])

'id'

(2028,)

'XC133548.mp3'

# **Random Forest Model**

Saving evaluation results

In [15]:
def evaluate_model(rf, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = rf.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = rf.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [16]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [17]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 126148)

In [18]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 126148)

Fit the model with training data

In [19]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [20]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['all_features'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['all_features'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2583826429980276


In [21]:
val_scores['all_features'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.28      0.17       126
           1       0.42      0.51      0.46       205
           2       0.12      0.34      0.18        96
           3       0.18      0.33      0.24       152
           4       0.09      0.03      0.05        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.41      0.45      0.43       107
           8       0.13      0.42      0.20        24
           9       0.17      0.31      0.22        32
          10       0.39      0.19      0.25        69
          11       0.14      0.00      0.01       272
          12       0.31      0.24      0.27       203
          13       0.45      0.07      0.11        76
          14       0.00      0.00      0.00        20
          15       0.10      0.13      0.12       119
          16       0.93      0.98      0.96   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7341690934739077
F1 Score (Weighted): 0.23695236709092815


### **With MFCCs only**

In [22]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 10340)

In [23]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 10340)

In [24]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [25]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.3032544378698225


In [26]:
val_scores['mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.28      0.65      0.40       126
           1       0.41      0.53      0.46       205
           2       0.17      0.64      0.26        96
           3       0.56      0.63      0.59       152
           4       0.00      0.00      0.00        33
           5       0.03      0.02      0.02        48
           6       0.00      0.00      0.00        38
           7       0.25      0.47      0.33       107
           8       0.08      0.08      0.08        24
           9       0.45      0.31      0.37        32
          10       0.17      0.03      0.05        69
          11       0.29      0.01      0.01       272
          12       0.46      0.38      0.42       203
          13       0.40      0.11      0.17        76
          14       0.02      0.05      0.03        20
          15       0.18      0.28      0.22       119
          16       0.54      0.47      0.50   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### **With Mel-Spectrogram Only**

In [27]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 66176)

In [28]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 66176)

In [29]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [30]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23323471400394477


In [31]:
val_scores['melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.16      0.32      0.21       126
           1       0.44      0.55      0.49       205
           2       0.15      0.32      0.20        96
           3       0.25      0.55      0.34       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.30      0.35      0.32       107
           8       0.09      0.38      0.15        24
           9       0.05      0.19      0.07        32
          10       0.36      0.17      0.24        69
          11       0.50      0.00      0.01       272
          12       0.26      0.15      0.19       203
          13       0.88      0.09      0.17        76
          14       0.00      0.00      0.00        20
          15       0.17      0.29      0.21       119
          16       0.33      0.20      0.25   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7225599076229777
F1 Score (Weighted): 0.20728561157132808


## **With Chroma Only**

In [32]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 6204)

In [33]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 6204)

In [34]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [35]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.16568047337278108


In [36]:
val_scores['chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.12      0.80      0.21       126
           1       0.29      0.44      0.35       205
           2       0.15      0.60      0.24        96
           3       0.45      0.07      0.11       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.10      0.22      0.14       107
           8       0.00      0.00      0.00        24
           9       0.00      0.00      0.00        32
          10       0.00      0.00      0.00        69
          11       0.00      0.00      0.00       272
          12       0.41      0.04      0.08       203
          13       0.00      0.00      0.00        76
          14       0.00      0.00      0.00        20
          15       0.34      0.25      0.29       119
          16       0.08      0.01      0.02   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **With CQT Only**

In [37]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 43428)

In [38]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 43428)

In [39]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [40]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.22781065088757396


In [41]:
val_scores['cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.34      0.20       126
           1       0.44      0.46      0.45       205
           2       0.08      0.28      0.13        96
           3       0.09      0.16      0.12       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.29      0.45      0.35       107
           8       0.09      0.04      0.06        24
           9       0.00      0.00      0.00        32
          10       0.00      0.00      0.00        69
          11       0.00      0.00      0.00       272
          12       0.32      0.30      0.31       203
          13       0.36      0.05      0.09        76
          14       0.00      0.00      0.00        20
          15       0.08      0.09      0.08       119
          16       0.93      0.97      0.95   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.6997099174059133
F1 Score (Weighted): 0.20793569281194837


## **With MFCCs and Mel-Spectrogram**

In [42]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 76516)

In [43]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 76516)

In [44]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [45]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2495069033530572


In [46]:
val_scores['mfcc_melspectrogram'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.34      0.21       126
           1       0.46      0.56      0.51       205
           2       0.15      0.38      0.22        96
           3       0.22      0.44      0.29       152
           4       0.10      0.03      0.05        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.27      0.32      0.29       107
           8       0.12      0.42      0.18        24
           9       0.10      0.19      0.13        32
          10       0.36      0.22      0.27        69
          11       0.00      0.00      0.00       272
          12       0.35      0.21      0.26       203
          13       0.71      0.07      0.12        76
          14       0.04      0.10      0.06        20
          15       0.17      0.25      0.21       119
          16       0.54      0.54      0.54   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7231582143526036
F1 Score (Weighted): 0.22334992578765328


## **With MFCCs and Chroma**

In [47]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 16544)

In [48]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 16544)

In [49]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [50]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2948717948717949


In [51]:
val_scores['mfcc_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.29      0.69      0.41       126
           1       0.40      0.46      0.43       205
           2       0.21      0.72      0.33        96
           3       0.49      0.64      0.56       152
           4       0.00      0.00      0.00        33
           5       0.07      0.06      0.07        48
           6       0.00      0.00      0.00        38
           7       0.25      0.47      0.33       107
           8       0.10      0.08      0.09        24
           9       0.31      0.34      0.32        32
          10       0.14      0.01      0.03        69
          11       0.29      0.01      0.01       272
          12       0.47      0.39      0.43       203
          13       0.30      0.11      0.16        76
          14       0.00      0.00      0.00        20
          15       0.16      0.25      0.20       119
          16       0.40      0.29      0.33   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **With MFCCs and CQT**

In [52]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 53768)

In [53]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 53768)

In [54]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [55]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['mfcc_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['mfcc_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2593688362919132


In [56]:
val_scores['mfcc_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.29      0.19       126
           1       0.46      0.54      0.50       205
           2       0.11      0.41      0.18        96
           3       0.17      0.30      0.22       152
           4       0.00      0.00      0.00        33
           5       0.03      0.02      0.03        48
           6       0.00      0.00      0.00        38
           7       0.34      0.45      0.38       107
           8       0.00      0.00      0.00        24
           9       0.35      0.25      0.29        32
          10       0.25      0.12      0.16        69
          11       0.00      0.00      0.00       272
          12       0.33      0.30      0.31       203
          13       0.50      0.09      0.16        76
          14       0.02      0.05      0.02        20
          15       0.11      0.13      0.12       119
          16       0.90      0.98      0.94   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7420490766788379
F1 Score (Weighted): 0.23508559193719317


## **With Chroma and CQT**

In [57]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 49632)

In [58]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 49632)

In [59]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [60]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['chroma_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['chroma_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23520710059171598


In [61]:
val_scores['chroma_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.17      0.42      0.25       126
           1       0.43      0.49      0.46       205
           2       0.08      0.27      0.12        96
           3       0.11      0.18      0.14       152
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.27      0.41      0.32       107
           8       0.11      0.08      0.10        24
           9       0.00      0.00      0.00        32
          10       0.08      0.01      0.02        69
          11       0.00      0.00      0.00       272
          12       0.31      0.29      0.30       203
          13       0.23      0.04      0.07        76
          14       0.00      0.00      0.00        20
          15       0.08      0.10      0.09       119
          16       0.89      0.97      0.93   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7139873160391721
F1 Score (Weighted): 0.20981916780392362


## **With Mel-Spectrogram and Chroma**

In [62]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 72380)

In [63]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 72380)

In [64]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [65]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.23126232741617356


In [66]:
val_scores['melspectrogram_chroma'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.14      0.29      0.19       126
           1       0.41      0.51      0.46       205
           2       0.17      0.38      0.23        96
           3       0.24      0.50      0.32       152
           4       0.17      0.03      0.05        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.29      0.36      0.32       107
           8       0.10      0.38      0.16        24
           9       0.05      0.16      0.08        32
          10       0.48      0.19      0.27        69
          11       0.00      0.00      0.00       272
          12       0.26      0.16      0.20       203
          13       1.00      0.03      0.05        76
          14       0.02      0.05      0.03        20
          15       0.13      0.22      0.16       119
          16       0.50      0.43      0.46   

## **With Mel-Spectrogram and CQT**

In [67]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 109604)

In [68]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 109604)

In [69]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [70]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.2514792899408284


In [71]:
val_scores['melspectrogram_cqt'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.07      0.14      0.10       126
           1       0.43      0.50      0.46       205
           2       0.14      0.36      0.21        96
           3       0.17      0.34      0.23       152
           4       0.11      0.03      0.05        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.44      0.45      0.45       107
           8       0.11      0.42      0.18        24
           9       0.09      0.19      0.12        32
          10       0.44      0.20      0.28        69
          11       0.00      0.00      0.00       272
          12       0.35      0.25      0.29       203
          13       0.50      0.04      0.07        76
          14       0.04      0.10      0.06        20
          15       0.10      0.14      0.11       119
          16       0.95      0.97      0.96   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7323292634258709
F1 Score (Weighted): 0.23142305795761492


## **With Mel-Spectrogram, MFCCs, CQT**

In [72]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 119944)

In [73]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 119944)

In [74]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [75]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.267258382642998


In [76]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.15      0.33      0.20       126
           1       0.43      0.55      0.48       205
           2       0.13      0.36      0.19        96
           3       0.19      0.32      0.24       152
           4       0.10      0.03      0.05        33
           5       0.03      0.02      0.03        48
           6       0.00      0.00      0.00        38
           7       0.42      0.46      0.44       107
           8       0.15      0.46      0.23        24
           9       0.11      0.19      0.13        32
          10       0.33      0.12      0.17        69
          11       0.20      0.00      0.01       272
          12       0.33      0.24      0.27       203
          13       0.46      0.08      0.13        76
          14       0.02      0.05      0.03        20
          15       0.11      0.14      0.12       119
          16       0.89      0.98      0.93   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


AUC Score: 0.7351230148544473
F1 Score (Weighted): 0.24076690220627206


## **With Mel-Spectrogram, MFCCs, Chroma**

In [77]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(6543, 82720)

In [78]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(2028, 82720)

In [79]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=1303)
rf.fit(training_features, train_y)

RandomForestClassifier(criterion='entropy', random_state=1303)

In [80]:
print('Training accuracy:', rf.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = rf.score(training_features, train_y)

print('Validation accuracy:', rf.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = rf.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.24358974358974358


In [81]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(rf=rf, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.17      0.36      0.23       126
           1       0.43      0.56      0.49       205
           2       0.15      0.38      0.21        96
           3       0.28      0.50      0.36       152
           4       0.09      0.03      0.05        33
           5       0.00      0.00      0.00        48
           6       0.00      0.00      0.00        38
           7       0.28      0.31      0.29       107
           8       0.12      0.46      0.19        24
           9       0.09      0.25      0.13        32
          10       0.28      0.14      0.19        69
          11       0.33      0.00      0.01       272
          12       0.34      0.24      0.28       203
          13       0.80      0.11      0.19        76
          14       0.02      0.05      0.03        20
          15       0.15      0.21      0.17       119
          16       0.43      0.32      0.37   

## Review results from all models

In [82]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.258
3,chroma,1.0,0.166
8,chroma_cqt,1.0,0.235
4,cqt,1.0,0.228
2,melspectrogram,1.0,0.233
9,melspectrogram_chroma,1.0,0.231
12,melspectrogram_chroma_mfcc,1.0,0.244
10,melspectrogram_cqt,1.0,0.251
11,melspectrogram_cqt_mfcc,1.0,0.267
1,mfcc,1.0,0.303


In [83]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(2)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features      0.24       0.73
3                    chroma      0.11       0.67
8                chroma_cqt      0.21       0.71
4                       cqt      0.21       0.70
2            melspectrogram      0.21       0.72
9     melspectrogram_chroma      0.20       0.72
10       melspectrogram_cqt      0.23       0.73
11  melspectrogram_cqt_mfcc      0.22       0.73
1                      mfcc      0.27       0.75
6               mfcc_chroma      0.25       0.75
7                  mfcc_cqt      0.24       0.74
5       mfcc_melspectrogram      0.22       0.72
